In [9]:
import psycopg2

def pg_get_conn(database="fakenews", user="fakenews", password="fnd"):
    """Get Postgres connection for fakenews

    Returns:
        Connection object : returns Post gres connection object

    Args:
        database (str, optional): Name of database
        user (str, optional): Name of User
        password (str, optional): Password of user
    """
    try:
        conn = psycopg2.connect(database=database,
                                user=user, password=password, host='localhost', port='5432')
        return conn
    except Exception as e:
        print(str(e))
        
def get_next_level_handles(rt=True):
    with pg_get_conn(database="abhishek",user="abhishek",password="vaishu") as conn:
        cur = conn.cursor()
        if rt:
            cur.execute("""Select retweeted_status_user_handle from tweet_articles_tweepy""")
        else:
            cur.execute("""Select * from tweet_articles_tweepy""")
        ans = cur.fetchall()
    print(ans)
    lse = set(x[0].replace("{","").replace("}","") for x in ans) if not rt else set(x[0] for x in ans)
    ans_handles=[]
    if not rt:
        for x in lse:
            for y in x.split(","):
                if y != '':
                    ans_handles.append(y)
    else:
        ans_handles = list(lse)
    return(ans_handles)

def write_next_handles(new_handles):
    path_old_file = '/home/abhishek/Documents/TweetCrawlMultiThreaded/handles.txt'
    old_handles = set()
    with open(path_old_file) as o_handles:
        for o_handle in o_handles:
            if o_handle.startswith('@'):
                o_handle = o_handle.replace('@','')
            old_handles.add(o_handle)
    new_handles = set(new_handles)
    next_level_handles = new_handles.difference(old_handles)
    with open(path_old_file+".bk", 'a') as f:
        for item in old_handles:
            f.write("%s\n" % item)
    with open(path_old_file, 'w') as f:
        for item in next_level_handles:
            f.write("%s\n" % item)

In [10]:
n_hand = get_next_level_handles(True)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
write_next_handles(n_hand)

In [75]:
len(set(ans_handles))

47868

6